In [1]:
import sys
print(sys.version)
print(spark.version)

3.8.5 | packaged by conda-forge | (default, Aug 29 2020, 01:22:49) 
[GCC 7.5.0]
3.0.1


In [2]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6 MB 5.4 MB/s eta 0:00:01
     |████████████████████████████████| 279 kB 20.6 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 6.2 MB/s  eta 0:00:01
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=de27258270468f2ac46f61201613e7ccd6bd05a511479b2efc373ad3efe8228e
  Stored in directory: /root/.cache/pip/wheels/31/8c/a0/24a443892f2134e691d59c8c6c35e19821e02f85e49871f8fd
Successfully built pyLDAvis


In [3]:
import pandas as pd
import numpy as np
from pyspark.sql.functions import *
from pyspark.sql.types import *
import seaborn as sns
import matplotlib.pyplot as plt
import pyLDAvis
warnings.filterwarnings(action='once')

In [4]:
import string
import pyspark
from nltk.corpus import stopwords
import re as re
from pyspark.mllib.linalg import Vector, Vectors, SparseVector
#from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.ml.clustering import LDA
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.mllib.util import MLUtils
from pyspark.ml.feature import CountVectorizer,  IDF, CountVectorizerModel, Tokenizer, RegexTokenizer, StopWordsRemover
import nltk
nltk.download('stopwords')

/opt/conda/miniconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/conda/miniconda3/lib/python3.8/site-packages/nltk/decorators.py:67: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  signature = inspect.formatargspec(
/opt/conda/miniconda3/lib/python3.8/site-packages/nltk/lm/counter.py:15: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  from collections import Sequence, defaultdict
/opt/conda/miniconda3/lib/python3.8/site-packages/nltk/lm/vocabulary.py:13: DeprecationWarning: Using or importing the 

True

In [5]:
sqlContext = SQLContext(sc)
# reading the data
df_uchi = sqlContext.read.format("csv") \
   .options(header='true', inferschema='true') \
   .load('gs://msca-bigdata-1/notebooks/gchai/uchicago_data1.csv')

df_unl=sqlContext.read.format("csv") \
   .options(header='true', inferschema='true') \
   .load('gs://msca-bigdata-1/notebooks/gchai/unebraska_data1.csv')

df_north=sqlContext.read.format("csv") \
   .options(header='true', inferschema='true') \
   .load('gs://msca-bigdata-1/notebooks/gchai/northeast_data1.csv')

df_brown=sqlContext.read.format("csv") \
   .options(header='true', inferschema='true') \
   .load('gs://msca-bigdata-1/notebooks/gchai/brown_data1.csv')

/usr/lib/spark/python/pyspark/sql/context.py:75: DeprecationWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


# Topic Modeling For Uchicago

In [6]:
#select the text column and do some pre-processing on the text: remove none data, remove stopwords, and split sentences
#into tokens.
uchi_text = df_uchi.rdd.map(lambda x : x['text']).filter(lambda x: x is not None)
StopWords = stopwords.words("english")
uchi_tokens = uchi_text\
    .map( lambda document: document.strip().lower())\
    .map( lambda document: re.split(" ", document))\
    .map( lambda word: [x for x in word if x.isalpha()])\
    .map( lambda word: [x for x in word if len(x) > 3] )\
    .map( lambda word: [x for x in word if x not in StopWords])\
    .zipWithIndex()

/opt/conda/miniconda3/lib/python3.8/socket.py:740: ResourceWarning: unclosed <socket.socket fd=60, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 44156), raddr=('127.0.0.1', 43585)>
  self._sock = None


In [7]:
#transform the rdd into a DataFrame which has two columns — one has index and the other the list of words. 
uchi_txts = sqlContext.createDataFrame(uchi_tokens, ["list_of_words",'id'])


/opt/conda/miniconda3/lib/python3.8/socket.py:740: ResourceWarning: unclosed <socket.socket fd=60, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 33956), raddr=('127.0.0.1', 45599)>
  self._sock = None


In [8]:
uchi_txts.show(5)

+--------------------+---+
|       list_of_words| id|
+--------------------+---+
|    [people, ignore]|  0|
|[four, uchicago, ...|  1|
|[professor, bruce...|  2|
|[congratulate, pr...|  3|
|[great, visit, st...|  4|
+--------------------+---+
only showing top 5 rows



### CountVectorizer takes this data and returns a sparse matrix of term frequencies attached to the original Dataframe. Same thing goes for the IDF.
* I use TFIDF feature for LDA model, because TFIDF will give less weight on words that have high frequency, in our case we know each university's name will appear a lot and we don't want to return the university name as the topic.

In [9]:
# TF
uchi_cv = CountVectorizer(inputCol="list_of_words", outputCol="features1", minDF=10.0)
uchi_cvmodel = uchi_cv.fit(uchi_txts)
result_cv1 = uchi_cvmodel.transform(uchi_txts)
# IDF
uchi_idf = IDF(inputCol="features1", outputCol="features")
uchi_idfModel = uchi_idf.fit(result_cv1)
result_tfidf1 = uchi_idfModel.transform(result_cv1) 

In [10]:
result_cv1.show(5)

+--------------------+---+--------------------+
|       list_of_words| id|           features1|
+--------------------+---+--------------------+
|    [people, ignore]|  0|(12977,[42,4110],...|
|[four, uchicago, ...|  1|(12977,[2,237,498...|
|[professor, bruce...|  2|(12977,[0,1,6,749...|
|[congratulate, pr...|  3|(12977,[319,1520,...|
|[great, visit, st...|  4|(12977,[3,37,403]...|
+--------------------+---+--------------------+
only showing top 5 rows



### Now, we have feature and index, let's prepare a dataframe to fit in the lda model
* I will chose 10 topics for each university
* Considering tweet text is short, I will limit each topic to 5 words

In [11]:
result1=result_tfidf1[['id','features']]
result1.show(5)

+---+--------------------+
| id|            features|
+---+--------------------+
|  0|(12977,[42,4110],...|
|  1|(12977,[2,237,498...|
|  2|(12977,[0,1,6,749...|
|  3|(12977,[319,1520,...|
|  4|(12977,[3,37,403]...|
+---+--------------------+
only showing top 5 rows



In [12]:
#define how many topics and iterations I want and start the training
num_topics = 10
max_iterations = 100
# Trains a LDA model
lda=LDA(k=num_topics, maxIter=max_iterations)
lda_model1 = lda.fit(result1)  
trans=lda_model1.transform(result1)

In [13]:
# Describe topics, chose 5 words for each topic
topics = lda_model1.describeTopics(5)
print("The topics described by their top-weighted terms:")
topics.show(truncate=False)

The topics described by their top-weighted terms:
+-----+----------------------+--------------------------------------------------------------------------------------------------------------+
|topic|termIndices           |termWeights                                                                                                   |
+-----+----------------------+--------------------------------------------------------------------------------------------------------------+
|0    |[30, 67, 38, 6, 0]    |[0.014622735529605147, 0.012755653106891817, 0.011719761083867653, 0.010855188166204049, 0.010095326248738445]|
|1    |[142, 25, 43, 90, 0]  |[0.009172123341861106, 0.008784090525029072, 0.008025932472311993, 0.007888283732278283, 0.007681728554157859]|
|2    |[20, 18, 40, 45, 0]   |[0.020688162714573303, 0.01702647252061524, 0.01635407573648827, 0.015404306210363837, 0.0150103422151199]    |
|3    |[1, 10, 14, 0, 52]    |[0.02728380076839592, 0.025410611534526402, 0.02413325216820956, 0.0

In [14]:
vocab = uchi_cvmodel.vocabulary 
topics_rdd = topics.rdd
topics_words = topics_rdd\
       .map(lambda row: row['termIndices'])\
       .map(lambda idx_list: [vocab[idx] for idx in idx_list])\
       .collect()
for idx, topic in enumerate(topics_words):
    print("topic: {}".format(idx))
    print("*"*25)
    t=[]
    for word in topic:
           t.append(word)
    print(t)
    print("*"*25)

topic: 0
*************************
['last', 'final', 'next', 'professor', 'university']
*************************
topic: 1
*************************
['days', 'campus', 'president', 'scientists', 'university']
*************************
topic: 2
*************************
['nobel', 'news', 'richard', 'prize', 'university']
*************************
topic: 3
*************************
['chicago', 'huge', 'dropping', 'university', 'american']
*************************
topic: 4
*************************
['medical', 'study', 'university', 'million', 'chicago']
*************************
topic: 5
*************************
['university', 'first', 'chicago', 'uchicago', 'anyone']
*************************
topic: 6
*************************
['turn', 'pressure', 'fluids', 'solids', 'chicago']
*************************
topic: 7
*************************
['uchicago', 'walk', 'students', 'south', 'chicago']
*************************
topic: 8
*************************
['uchicago', 'position', 'hiring', 

/opt/conda/miniconda3/lib/python3.8/socket.py:740: ResourceWarning: unclosed <socket.socket fd=60, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 55658), raddr=('127.0.0.1', 38415)>
  self._sock = None


## Now, Let's visualize the topics. 
### Before we visualize the topics, we will need to prepare some data for pyLDAvis, which is a visualization tool for topic modeling.
We need the following data
* topic_term_dists : array-like, shape (n_topics, n_terms)
        Matrix of topic-term probabilities. Where n_terms is len(vocab).
* doc_topic_dists : array-like, shape (n_docs, n_topics)
        Matrix of document-topic probabilities.
* doc_lengths : array-like, shape n_docs
        The length of each document, i.e. the number of words in each document.
        The order of the numbers should be consistent with the ordering of the
        docs in doc_topic_dists.
* vocab : array-like, shape n_terms
        List of all the words in the corpus used to train the model.
* term_frequency : array-like, shape n_terms
        The count of each particular term over the entire corpus. The ordering
        of these counts should correspond with `vocab` and topic_term_dists.


In [15]:
# prepare data
def format_data_to_pyldavis(df, count_vectorizer, transformed, lda_model):
    xxx = df.select((explode(df.list_of_words)).alias("words")).groupby("words").count()
    word_counts = {r['words']:r['count'] for r in xxx.collect()}
    word_counts = [word_counts[w] for w in count_vectorizer.vocabulary]


    data = {'topic_term_dists': np.array(lda_model.topicsMatrix().toArray()).T, 
            'doc_topic_dists': np.array([x.toArray() for x in transformed.select(["topicDistribution"]).toPandas()['topicDistribution']]),
            'doc_lengths': [r[0] for r in df.select(size(df.list_of_words)).collect()],
            'vocab': count_vectorizer.vocabulary,
            'term_frequency': word_counts}

    return data

In [16]:
# for some reason some docs apears with 0 value in all the vectors, or the norm is not 1, so I filter out those docs.
def filter_bad_docs(data):
    bad = 0
    doc_topic_dists_filtrado = []
    doc_lengths_filtrado = []

    for x,y in zip(data['doc_topic_dists'], data['doc_lengths']):
        if np.sum(x)==0:
            bad+=1
        elif np.sum(x) != 1:
            bad+=1
        elif np.isnan(x).any():
            bad+=1
        else:
            doc_topic_dists_filtrado.append(x)
            doc_lengths_filtrado.append(y)

    data['doc_topic_dists'] = doc_topic_dists_filtrado
    data['doc_lengths'] = doc_lengths_filtrado


In [17]:
data = format_data_to_pyldavis(uchi_txts, uchi_cvmodel, trans, lda_model1)

/opt/conda/miniconda3/lib/python3.8/socket.py:740: ResourceWarning: unclosed <socket.socket fd=60, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 53506), raddr=('127.0.0.1', 40305)>
  self._sock = None
/opt/conda/miniconda3/lib/python3.8/socket.py:740: ResourceWarning: unclosed <socket.socket fd=60, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 49124), raddr=('127.0.0.1', 43385)>
  self._sock = None
/opt/conda/miniconda3/lib/python3.8/socket.py:740: ResourceWarning: unclosed <socket.socket fd=60, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 48426), raddr=('127.0.0.1', 42785)>
  self._sock = None


## Now, let's visualize our topics
* In the pyLDAvis graph, the area of circle represents the importance of each topic over the entire corpus, 
the distance between the center of circles indicate the similarity between topics. 
* For each topic, the histogram on the right side listed the top 30 most relevant terms. 

In [18]:
filter_bad_docs(data)
py_lda_prepared_data = pyLDAvis.prepare(**data)
pyLDAvis.display(py_lda_prepared_data)

## UChicago Summary: 
LDA model usese bag-of-words, which means it doesn't consider grammae. Thus, the result only gives us the brief idea that what topics those tweets are mentioning. Based on the LDA resuly, the most frequently discussed topics for UChicago relates to: professor, dropping, nobel prize, grad, researcher, data, final, pressure, hiring, spring. 

# Topic Modeling for UNL

In [19]:
#select the text column and do some pre-processing on the text: remove none data, remove stopwords, and split sentences
#into tokens.
unl_text = df_unl.rdd.map(lambda x : x['text']).filter(lambda x: x is not None)
StopWords = stopwords.words("english")
unl_tokens = unl_text\
    .map( lambda document: document.strip().lower())\
    .map( lambda document: re.split(" ", document))\
    .map( lambda word: [x for x in word if x.isalpha()])\
    .map( lambda word: [x for x in word if len(x) > 3] )\
    .map( lambda word: [x for x in word if x not in StopWords])\
    .zipWithIndex()

/opt/conda/miniconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/conda/miniconda3/lib/python3.8/socket.py:740: ResourceWarning: unclosed <socket.socket fd=79, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 33110), raddr=('127.0.0.1', 44671)>
  self._sock = None


In [20]:
#transform the rdd into a DataFrame which has two columns — one has index and the other the list of words. 
unl_txts = sqlContext.createDataFrame(unl_tokens, ["list_of_words",'id'])


/opt/conda/miniconda3/lib/python3.8/socket.py:740: ResourceWarning: unclosed <socket.socket fd=79, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 56416), raddr=('127.0.0.1', 38647)>
  self._sock = None


### CountVectorizer: TFIDF

In [21]:
# TF
unl_cv = CountVectorizer(inputCol="list_of_words", outputCol="features1", minDF=10.0)
unl_cvmodel = unl_cv.fit(unl_txts)
result_cv2 = unl_cvmodel.transform(unl_txts)
# IDF
unl_idf = IDF(inputCol="features1", outputCol="features")
unl_idfModel = unl_idf.fit(result_cv2)
result_tfidf2 = unl_idfModel.transform(result_cv2) 
result2=result_tfidf2[['id','features']]

### Fit LDA Model

In [22]:
#define how many topics and iterations I want and start the training
num_topics = 10
max_iterations = 100
# Trains a LDA model
lda=LDA(k=num_topics, maxIter=max_iterations)
lda_model2 = lda.fit(result2)  
trans2=lda_model2.transform(result2)

### Check Topic Result

In [23]:
# Describe topics, chose 5 words for each topic
topics2 = lda_model2.describeTopics(5)
print("The topics described by their top-weighted terms:")
topics2.show(truncate=False)

The topics described by their top-weighted terms:
+-----+----------------------+-------------------------------------------------------------------------------------------------------------+
|topic|termIndices           |termWeights                                                                                                  |
+-----+----------------------+-------------------------------------------------------------------------------------------------------------+
|0    |[20, 46, 55, 50, 60]  |[0.05465053589501281, 0.032629265511294595, 0.026704990289155198, 0.026307383590175332, 0.024627849522391766]|
|1    |[31, 37, 54, 97, 92]  |[0.047585377765290655, 0.04019778567859959, 0.026860447510390927, 0.021608384961975338, 0.021535663145174803]|
|2    |[7, 6, 12, 24, 30]    |[0.052522333317348076, 0.045604568279567916, 0.03907746379755913, 0.03714969276535672, 0.031618365786037696] |
|3    |[2, 3, 4, 5, 8]       |[0.09665514053797059, 0.09591188383938136, 0.09368752433376133, 0.07266341

/usr/lib/spark/python/pyspark/sql/context.py:117: DeprecationWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [24]:
vocab2 = unl_cvmodel.vocabulary 
topics2_rdd = topics2.rdd
topics2_words = topics2_rdd\
       .map(lambda row: row['termIndices'])\
       .map(lambda idx_list: [vocab2[idx] for idx in idx_list])\
       .collect()
for idx, topic in enumerate(topics2_words):
    print("topic: {}".format(idx))
    print("*"*25)
    t=[]
    for word in topic:
           t.append(word)
    print(t)
    print("*"*25)

topic: 0
*************************
['lyft', 'another', 'code', 'zoot', 'offered']
*************************
topic: 1
*************************
['grateful', 'humbled', 'like', 'would', 'congrats']
*************************
topic: 2
*************************
['announce', 'excited', 'committed', 'academic', 'career']
*************************
topic: 3
*************************
['offer', 'blessed', 'receive', 'received', 'honored']
*************************
topic: 4
*************************
['coach', 'visit', 'good', 'great', 'omaha']
*************************
topic: 5
*************************
['professor', 'associate', 'office', 'congressman', 'called']
*************************
topic: 6
*************************
['thankful', 'walk', 'honor', 'represent', 'remarkable']
*************************
topic: 7
*************************
['student', 'president', 'chancellor', 'last', 'already']
*************************
topic: 8
*************************
['truly', 'scholarship', 'beyond', 'offic

/opt/conda/miniconda3/lib/python3.8/socket.py:740: ResourceWarning: unclosed <socket.socket fd=79, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 32958), raddr=('127.0.0.1', 41331)>
  self._sock = None


### Visualizing Topics

In [25]:
data2 = format_data_to_pyldavis(unl_txts, unl_cvmodel, trans2, lda_model2)
filter_bad_docs(data2)
py_lda_prepared_data = pyLDAvis.prepare(**data2)
pyLDAvis.display(py_lda_prepared_data)

/opt/conda/miniconda3/lib/python3.8/socket.py:740: ResourceWarning: unclosed <socket.socket fd=79, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 43070), raddr=('127.0.0.1', 46065)>
  self._sock = None
/opt/conda/miniconda3/lib/python3.8/socket.py:740: ResourceWarning: unclosed <socket.socket fd=79, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 43798), raddr=('127.0.0.1', 44453)>
  self._sock = None
/opt/conda/miniconda3/lib/python3.8/socket.py:740: ResourceWarning: unclosed <socket.socket fd=79, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 59124), raddr=('127.0.0.1', 35431)>
  self._sock = None


## UNL Summary: 
Based on the LDA resuly, the most frequently discussed topics for UNL relates to: congratulations, academic, career, offer, coach, professor, chancellor, students, cornhusker, medical.

# Topic Modeling for Northeastern University

In [26]:
#select the text column and do some pre-processing on the text: remove none data, remove stopwords, and split sentences
#into tokens.
north_text = df_north.rdd.map(lambda x : x['text']).filter(lambda x: x is not None)
StopWords = stopwords.words("english")
north_tokens = north_text\
    .map( lambda document: document.strip().lower())\
    .map( lambda document: re.split(" ", document))\
    .map( lambda word: [x for x in word if x.isalpha()])\
    .map( lambda word: [x for x in word if len(x) > 3] )\
    .map( lambda word: [x for x in word if x not in StopWords])\
    .zipWithIndex()

/opt/conda/miniconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/conda/miniconda3/lib/python3.8/socket.py:740: ResourceWarning: unclosed <socket.socket fd=79, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 55762), raddr=('127.0.0.1', 45571)>
  self._sock = None


In [27]:
#transform the rdd into a DataFrame which has two columns — one has index and the other the list of words. 
north_txts = sqlContext.createDataFrame(north_tokens, ["list_of_words",'id'])

/opt/conda/miniconda3/lib/python3.8/socket.py:740: ResourceWarning: unclosed <socket.socket fd=79, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 38818), raddr=('127.0.0.1', 39997)>
  self._sock = None


### CountVectorizer: TFIDF

In [28]:
# TF
north_cv = CountVectorizer(inputCol="list_of_words", outputCol="features1", minDF=10.0)
north_cvmodel = north_cv.fit(north_txts)
result_cv3 = north_cvmodel.transform(north_txts)
# IDF
north_idf = IDF(inputCol="features1", outputCol="features")
north_idfModel = north_idf.fit(result_cv3)
result_tfidf3 = north_idfModel.transform(result_cv3) 
result3=result_tfidf3[['id','features']]

### Fit LDA Model

In [29]:
#define how many topics and iterations I want and start the training
num_topics = 10
max_iterations = 100
# Trains a LDA model
lda=LDA(k=num_topics, maxIter=max_iterations)
lda_model3 = lda.fit(result3)  
trans3=lda_model3.transform(result3)

### Check Topic Results

In [30]:
# Describe topics, chose 5 words for each topic
topics3 = lda_model3.describeTopics(5)
print("The topics described by their top-weighted terms:")
topics3.show(truncate=False)

The topics described by their top-weighted terms:
+-----+--------------------+-------------------------------------------------------------------------------------------------------------+
|topic|termIndices         |termWeights                                                                                                  |
+-----+--------------------+-------------------------------------------------------------------------------------------------------------+
|0    |[43, 82, 83, 49, 95]|[0.030613342466282452, 0.02375755481232285, 0.023324083888331338, 0.023195580839153462, 0.017600856825064685]|
|1    |[42, 14, 62, 87, 98]|[0.034470716600754424, 0.030927600468784774, 0.02264491689304544, 0.021141877299074463, 0.020954050522176717]|
|2    |[8, 6, 10, 9, 3]    |[0.10932158823195577, 0.10931440828771516, 0.10918162944360264, 0.10918156541902621, 0.1049572608472929]     |
|3    |[21, 37, 15, 55, 56]|[0.035143300842689226, 0.031224329751311328, 0.025507655816001583, 0.02517487279043652, 

/usr/lib/spark/python/pyspark/sql/context.py:117: DeprecationWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [31]:
vocab3 = north_cvmodel.vocabulary 
topics3_rdd = topics3.rdd
topics3_words = topics3_rdd\
       .map(lambda row: row['termIndices'])\
       .map(lambda idx_list: [vocab3[idx] for idx in idx_list])\
       .collect()
for idx, topic in enumerate(topics3_words):
    print("topic: {}".format(idx))
    print("*"*25)
    t=[]
    for word in topic:
           t.append(word)
    print(t)
    print("*"*25)

topic: 0
*************************
['sunset', 'event', 'know', 'says', 'pelosi']
*************************
topic: 1
*************************
['sunrise', 'team', 'program', 'director', 'together']
*************************
topic: 2
*************************
['applied', 'admitted', 'reassure', 'disciplinary', 'want']
*************************
topic: 3
*************************
['professor', 'associate', 'boston', 'jennifer', 'bowen']
*************************
topic: 4
*************************
['miles', 'wind', 'brisk', 'speed', 'windy']
*************************
topic: 5
*************************
['study', 'announce', 'excited', 'harvard', 'majority']
*************************
topic: 6
*************************
['violating', 'social', 'dismissed', 'caught', 'distancing']
*************************
topic: 7
*************************
['according', 'love', 'dogs', 'humans', 'researchers']
*************************
topic: 8
*************************
['health', 'school', 'public', 'weather',

/opt/conda/miniconda3/lib/python3.8/socket.py:740: ResourceWarning: unclosed <socket.socket fd=80, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 56574), raddr=('127.0.0.1', 33655)>
  self._sock = None


### Visualizing Topics

In [32]:
data3 = format_data_to_pyldavis(north_txts, north_cvmodel, trans3, lda_model3)
filter_bad_docs(data3)
py_lda_prepared_data = pyLDAvis.prepare(**data3)
pyLDAvis.display(py_lda_prepared_data)

/opt/conda/miniconda3/lib/python3.8/socket.py:740: ResourceWarning: unclosed <socket.socket fd=80, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 55534), raddr=('127.0.0.1', 37289)>
  self._sock = None
/opt/conda/miniconda3/lib/python3.8/socket.py:740: ResourceWarning: unclosed <socket.socket fd=80, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 58386), raddr=('127.0.0.1', 34285)>
  self._sock = None
/opt/conda/miniconda3/lib/python3.8/socket.py:740: ResourceWarning: unclosed <socket.socket fd=80, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 34126), raddr=('127.0.0.1', 38133)>
  self._sock = None


## Northeastern Summary:
Based on the LDA resuly, the most frequently discussed topics for Northeastern relates to: library, wind, boston, tuition, student,  professor, basketball,
offer, harvard, dismissed, dogs, weather.


# Topic Modeling for Brown University

In [33]:
#select the text column and do some pre-processing on the text: remove none data, remove stopwords, and split sentences
#into tokens.
brown_text = df_brown.rdd.map(lambda x : x['text']).filter(lambda x: x is not None)
StopWords = stopwords.words("english")
brown_tokens = brown_text\
    .map( lambda document: document.strip().lower())\
    .map( lambda document: re.split(" ", document))\
    .map( lambda word: [x for x in word if x.isalpha()])\
    .map( lambda word: [x for x in word if len(x) > 3] )\
    .map( lambda word: [x for x in word if x not in StopWords])\
    .zipWithIndex()

/opt/conda/miniconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/conda/miniconda3/lib/python3.8/socket.py:740: ResourceWarning: unclosed <socket.socket fd=80, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 35886), raddr=('127.0.0.1', 34245)>
  self._sock = None


In [34]:
#transform the rdd into a DataFrame which has two columns — one has index and the other the list of words. 
brown_txts = sqlContext.createDataFrame(brown_tokens, ["list_of_words",'id'])

/opt/conda/miniconda3/lib/python3.8/socket.py:740: ResourceWarning: unclosed <socket.socket fd=80, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 35022), raddr=('127.0.0.1', 39755)>
  self._sock = None


### CountVectorizer: TFIDF

In [35]:
# TF
brown_cv = CountVectorizer(inputCol="list_of_words", outputCol="features1", minDF=10.0)
brown_cvmodel = brown_cv.fit(brown_txts)
result_cv4 = brown_cvmodel.transform(brown_txts)
# IDF
brown_idf = IDF(inputCol="features1", outputCol="features")
brown_idfModel = brown_idf.fit(result_cv4)
result_tfidf4 = brown_idfModel.transform(result_cv4) 
result4=result_tfidf4[['id','features']]

### Fit LDA Model

In [36]:
#define how many topics and iterations I want and start the training
num_topics = 10
max_iterations = 100
# Trains a LDA model
lda=LDA(k=num_topics, maxIter=max_iterations)
lda_model4 = lda.fit(result4)  
trans4=lda_model4.transform(result4)

### Check Topic Results

In [37]:
# Describe topics, chose 5 words for each topic
topics4 = lda_model4.describeTopics(5)
print("The topics described by their top-weighted terms:")
topics4.show(truncate=False)

The topics described by their top-weighted terms:
+-----+------------------------+-------------------------------------------------------------------------------------------------------------+
|topic|termIndices             |termWeights                                                                                                  |
+-----+------------------------+-------------------------------------------------------------------------------------------------------------+
|0    |[4, 3, 6, 9, 10]        |[0.0918240126506903, 0.06066736485665482, 0.04920511632347541, 0.047142311154897885, 0.04660509396871877]    |
|1    |[8, 27, 30, 50, 46]     |[0.04411598833539572, 0.029661185697420097, 0.0274386694868953, 0.022593897081121164, 0.0216205711076205]    |
|2    |[42, 47, 49, 132, 116]  |[0.02684959985470113, 0.026220910390648916, 0.025501335800798188, 0.01730163872868624, 0.016290665685016827] |
|3    |[14, 19, 21, 22, 24]    |[0.04590611664428574, 0.04153455611038194, 0.041194784855712

/usr/lib/spark/python/pyspark/sql/context.py:117: DeprecationWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [38]:
vocab4 = brown_cvmodel.vocabulary 
topics4_rdd = topics4.rdd
topics4_words = topics4_rdd\
       .map(lambda row: row['termIndices'])\
       .map(lambda idx_list: [vocab4[idx] for idx in idx_list])\
       .collect()
for idx, topic in enumerate(topics4_words):
    print("topic: {}".format(idx))
    print("*"*25)
    t=[]
    for word in topic:
           t.append(word)
    print(t)
    print("*"*25)

topic: 0
*************************
['postdoctoral', 'know', 'rhode', 'fellowship', 'united']
*************************
topic: 1
*************************
['offer', 'blessed', 'receive', 'received', 'excited']
*************************
topic: 2
*************************
['click', 'want', 'work', 'hiring', 'link']
*************************
topic: 3
*************************
['still', 'institution', 'minutes', 'eighth', 'existed']
*************************
topic: 4
*************************
['school', 'health', 'dean', 'public', 'programs']
*************************
topic: 5
*************************
['football', 'message', 'gives', 'morning', 'coordinator']
*************************
topic: 6
*************************
['students', 'order', 'spirited', 'free', 'education']
*************************
topic: 7
*************************
['students', 'plans', 'life', 'plan', 'printing']
*************************
topic: 8
*************************
['study', 'researcher', 'university', 'released'

/opt/conda/miniconda3/lib/python3.8/socket.py:740: ResourceWarning: unclosed <socket.socket fd=80, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 53082), raddr=('127.0.0.1', 44097)>
  self._sock = None


### Visualizing Topics

In [39]:
data4 = format_data_to_pyldavis(brown_txts, brown_cvmodel, trans4, lda_model4)
filter_bad_docs(data4)
py_lda_prepared_data = pyLDAvis.prepare(**data4)
pyLDAvis.display(py_lda_prepared_data)

/opt/conda/miniconda3/lib/python3.8/socket.py:740: ResourceWarning: unclosed <socket.socket fd=80, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 45760), raddr=('127.0.0.1', 46267)>
  self._sock = None
/opt/conda/miniconda3/lib/python3.8/socket.py:740: ResourceWarning: unclosed <socket.socket fd=80, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 38992), raddr=('127.0.0.1', 35403)>
  self._sock = None
/opt/conda/miniconda3/lib/python3.8/socket.py:740: ResourceWarning: unclosed <socket.socket fd=80, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 44568), raddr=('127.0.0.1', 46537)>
  self._sock = None


## Brown Summary:
Based on the LDA resuly, the most frequently discussed topics for Brown relates to: postdoctoral, offer, hiring, health, football, student, spirited, vegan, researcher, league.